In [1]:
import numpy as np
import pandas as pd

One-hot encoding anime sources and genres

In [2]:
data_anime = pd.read_csv('Data/Animelist.csv')

In [3]:
tv = data_anime[data_anime['type'] == 'TV']

In [4]:
genres = tv['genre'].str.get_dummies(sep=', ')
genre_encoded = pd.concat([tv, genres], axis=1)

In [5]:
source_encoded = pd.get_dummies(genre_encoded['source'], prefix='source').astype(int)
data_filtered = pd.concat([genre_encoded, source_encoded], axis=1)

In [6]:
data_filtered

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,source_Manga,source_Music,source_Novel,source_Original,source_Other,source_Picture book,source_Radio,source_Unknown,source_Visual novel,source_Web manga
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,...,1,0,0,0,0,0,0,0,0,0
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,1,0,0,0,0,0,0,0,0,0
2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,...,1,0,0,0,0,0,0,0,0,0
3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,...,0,0,0,1,0,0,0,0,0,0
4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,Finished Airing,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14414,37662,Shinya! Tensai Bakabon,NaN,深夜! 天才バカボン,Late Night! Genius Bakabon,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,0,Not yet aired,...,1,0,0,0,0,0,0,0,0,0
14431,37300,Xiao Li Yu Li Xian Ji,The Adventures of Little Carp,小鲤鱼历险记,"小鲤鱼历险记,",https://myanimelist.cdn-dena.com/images/anime/...,TV,Other,52,Finished Airing,...,0,0,0,0,1,0,0,0,0,0
14442,37428,Chara to Otamajakushi Shima,NaN,キャラとおたまじゃくし島,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,0,Currently Airing,...,0,0,0,1,0,0,0,0,0,0
14463,37787,Space Bug,NaN,スペースバグ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,0,Not yet aired,...,0,0,0,1,0,0,0,0,0,0


Filtering users to avoid fake accounts

In [7]:
data_filtered.to_csv('Data/animelist_filt.csv')

In [8]:
data_users = pd.read_csv('Data/UserList.csv')
data_users.dropna(subset = ['birth_date', 'location', 'gender'], inplace=True)

In [9]:
data_users.to_csv('Data/userlist_filt.csv')
data_users

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,access_rank,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes
0,karthiga,2255153,3,49,1,0,0,55.31,Female,"Chennai, India",1990-04-29,NaN,2013-03-03,2014-02-04 01:32:00,7.43,0.0,3391.0
1,RedvelvetDaisuki,1897606,61,396,39,0,206,118.07,Female,Manila,1995-01-01,NaN,2012-12-13,1900-05-13 02:47:00,6.78,80.0,7094.0
2,Damonashu,37326,45,195,27,25,59,83.70,Male,"Detroit,Michigan",1991-08-01,NaN,2008-02-13,1900-03-24 12:48:00,6.15,6.0,4936.0
3,bskai,228342,25,414,2,5,11,167.16,Male,"Nayarit, Mexico",1990-12-14,NaN,2009-08-31,2014-05-12 16:35:00,8.27,1.0,10081.0
5,terune_uzumaki,327311,5,5,0,0,0,15.20,Female,"Malaysia, Kuantan",1998-08-24,NaN,2010-05-10,2012-10-18 19:06:00,9.70,6.0,920.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302652,bumcakee,1396285,13,116,5,9,22,49.87,Female,"nhollywood, california",1998-07-18,NaN,2012-07-14,2017-02-14 00:30:00,8.86,24.0,3027.0
302655,ammaretto,381078,0,1,1,0,0,0.49,Male,"Russia, Novokuznetsk",1990-08-02,NaN,2010-09-09,2015-06-03 10:59:00,8.00,0.0,27.0
302657,Scarlet95,2478991,6,103,10,8,54,47.94,Female,Belgium,1995-10-17,NaN,2013-04-24,2016-12-18 08:41:00,7.40,1.0,2906.0
302671,Torasori,3975907,22,239,0,4,176,86.88,Male,"Latvia, Riga",1998-11-18,NaN,2014-07-30,2018-05-24 21:34:46,8.98,47.0,5313.0


Making sure to drop the lists of filtered users
###### this code may take a few minutes to run

In [10]:
unique_users = data_users['username'].unique()
data_user_anime = pd.read_csv('Data/UserAnimeList.csv')
data_user_anime_filtered = data_user_anime[data_user_anime['username'].isin(unique_users)]

In [11]:
data_user_anime_filtered.to_csv('Data/userAnimeList_filt.csv')